# DrivenData - Genetic Engineering Attribution Challenge
#### 12-07-2020, Jim set up Benchmark
#### 12-10-2020, Vanessa added dataset links 
#### 12-19-2020, Group Hackthon collaborated efforts with RFC, Native Bayes,
+ RBF and Neuro Network In progress (session 420 and 430)

#### future work
+ impute N in the DNA sequence
    - Idea 1 during the n-gram feature generation phase, add the N related feature occurance
+ implement the ability to show progress of space search
+ implement the ability to generate and eliminate features
+ implement test out RBF and Neuron Network estimator.
    - use guassian process for RBF?

#### additional functionality extension
+ figure out how to premutate more than 5 n-gram

In [ ]:
### Import Libraries

from pathlib import Path

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import time

%matplotlib inline
sns.set()

print("Libraries Loaded")

## 100 Read Data

In [ ]:
### Following code for reading off of JMO Harddrive

#DATA_DIR = 'C:/Users/jorrison/Documents/Home/Harvard/DataMinds/Genetic Engineering/Data/'

#train_values = pd.read_csv(DATA_DIR + 'train_values.csv', index_col='sequence_id')
#train_labels = pd.read_csv(DATA_DIR + 'train_labels.csv', index_col='sequence_id')
#test_values  = pd.read_csv(DATA_DIR + 'test_values.csv' , index_col='sequence_id')

#train_values.head()

In [ ]:
### Read local by roland

train_labels = pd.read_csv('train_labels.csv', index_col='sequence_id')
train_values = pd.read_csv('train_values.csv', index_col='sequence_id')

In [ ]:
### remote read

#train_values = pd.read_csv('https://www.dropbox.com/s/3atnfvr65oh0y4y/train_values.csv?dl=1', index_col='sequence_id')
#train_labels = pd.read_csv('https://www.dropbox.com/s/71yg5f3dbc4h2yd/train_labels.csv?dl=1', index_col='sequence_id')
#test_values  = pd.read_csv('https://www.dropbox.com/s/4g1astwaccer5lz/test_values.csv?dl=1' , index_col='sequence_id')

# train_values.head()

### 110 Explore the Training Data

In [ ]:
train_values.dtypes

### 120 remove the lab has the highest frequency occurance in dataset.

largelab_rownum = train_labels[ train_labels['I7FXTVDP'] > 0 ].index
largelab_rownum.shape
largelab_rownum
train_labels_dropped = train_labels.drop(largelab_rownum)
train_values_dropped = train_values.drop(largelab_rownum)
train_labels_dropped.shape
train_values_dropped.shape

### 130 reduce the output labels dimension to string label text.
#lab_ids = pd.DataFrame(train_labels.idxmax(axis=1), columns = ['lab_id'])
lab_ids = pd.DataFrame(train_labels_dropped.idxmax(axis=1), columns = ['lab_id'])

In [ ]:
sequence_lengths = train_values.sequence.apply(len)

sequence_lengths.describe()

### 140 visualize number of sequence code with length of 2500 or less.
sequence_lengths.plot(kind='hist',
                     title= 'Distribution of DNA Sequence Length', 
                     bins = 2500,
                     xlim = (0, 2500))   # Suppress a bar longer than 20,000

### 150 visualize number of genetic sequence code with length of 1700 or less
sequence_lengths[sequence_lengths<1700].plot(kind='hist',
                     title= 'Distribution of DNA Sequence Length', 
                     bins = 100)

### 160 extrapolate data with genetic sequence length 1700 or less

index_ferdous = sequence_lengths[sequence_lengths<1700].index
df_ferdous_values = train_values_dropped.loc[index_ferdous]

### 170 visualize the value occurance of other 39 features contained in training values.

In [ ]:
sorted_binary_features = train_values.iloc[:, 1:].mean().sort_values()

ax = sorted_binary_features.plot(kind = 'barh',
                                stacked = True,
                                figsize = (5,12),
                                title   = 'Prevalence of Binary Features')
ax.set_xlabel('Proportion of sequences')

### 175 inspect the output lab occurrance

In [ ]:
train_labels.sum().describe()

### 180 Explore the Test Data

skipped, since we can't check the test data result

### 190 Explore Training Labels

In [ ]:
train_labels.head()

In [ ]:
# Get the column with the max value in each row

lab_ids = pd.DataFrame(train_labels.idxmax(axis=1), columns = ['lab_id'])
lab_ids

In [ ]:
# Get the distribution of lab prevalence in the training set
lab_ids['lab_id'].value_counts().describe()

In [ ]:
# Sort lab ids by prevalence
(lab_ids['lab_id'].value_counts(normalize = True).sort_values(ascending=False)).head()

In [ ]:
#lab_ids_ferdous = lab_ids.loc[index_ferdous]

### 200 Enrich Data Sets
#### 210 Construct Features from DNA Sequences

In [ ]:
#bases = set(''.join(train_values.sequence.values))
bases ={'A', 'C', 'G', 'N', 'T'}
bases

In [ ]:
from itertools import permutations

n = 5
subsequences = [''.join(permutation) for permutation in permutations(bases, r=n)]

len(subsequences)

#### 220 Reduced Permutation to test out the effect. seems the 2-gram approach is not as good as 4 grams
n = 2
subsequences = [''.join(permutation) for permutation in permutations(bases, r=n)]
print(f"Number of subsequences: {len(subsequences)}")
subsequences[:10]

#### 230 Example of built-in count method on strings
Because it's non-overlapping, "atta" is only counted twice
"gattattattaca".count("atta")

#### 240 Counting the Subsequences

In [ ]:
def get_ngram_features(data, subsequences):
    """Generates counts for each subsequence.
    
    Args:
        data (DataFrame): The data you want to create features from. Must subsequences (list): A list of subsequences to count
        
    Returns:
        DataFrame: A DataFrame with one column for each subsequence
    """
    
    features = pd.DataFrame(index=data.index)
    for subseq in subsequences:
        features[subseq] = data.sequence.str.count(subseq)
    return features

In [ ]:
# Calculate n-gram features on our training set (ferdous training set)
ngram_features = get_ngram_features(train_values, subsequences)
ngram_features.head()

In [ ]:
ngram_features.value_counts().describe()

# Calculate n-gram features on our training set
ngram_features = get_ngram_features(train_values, subsequences)
ngram_features.head()
ngram_features.shape

In [ ]:
all_features = ngram_features.join(train_values.drop('sequence', axis=1))

all_features

In [ ]:
all_features.shape

In [ ]:
from sklearn.preprocessing import StandardScaler

default_scaler = StandardScaler()
default_scaler.fit(all_features)
all_features_scaled = default_scaler.transform(all_features)


### 300 Processing Top Ten Metric

In [ ]:
def top10_accuracy_scorer(estimator, X, y):
    """A custom scorer that evaluates a model on whether the correct label the top 10 most probably predictions.
    
    Args:
        estimatory (sklearn estimator): The skLearn model that should be evaluatioed 
        X (numpy array): The validation data.
        y (numpy array): The ground truth lables.
        
    Returns:
        float: Accuracy of the model as defined by the proportion of prediction in which the correct label was in Top 10
    """
    # predict the probabilities across all possible labels for rows in our training set
    probas = estimator.predict_proba(X)
    
    # get the indices for top 10 predictions for each row; these are these are the last ten in each row
    # Note: We use argpartition, which is O(n), vs argsort, which uses the quicksort algorithm 
    # by default and is O(n^2) in the worst case. We can do this because we only need the top ten
    # partitioned, not in sorted order.
    
    # Documentation: https://numpy.org/doc/1.18/reference/generated/numpy.argpartition.html
    top10_idx = np.argpartition(probas, -10, axis=1)[:, -10:]
    
    # index into the classes list using the top ten indices to get the class names
    top10_preds = estimator.classes_[top10_idx]

    # check if y-true is in top 10 for each set of predictions
    mask = top10_preds == y.reshape((y.size, 1))
    
    # take the mean
    top_10_accuracy = mask.any(axis=1).mean()
 
    return top_10_accuracy

### 350 Break apart Training Data Set into Train-Validate-Test

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
# Train_test_split will generally split the data set as per describe the test_size
# However under the condition when there is only one set of data available, train_test_split will return an error.
# for quickSplit, it will solve the issue as per general train_test_split except for the single record,
# which quickSplit will return the exact value back as train and test.

def quickSplit(X,y, TEST_SIZE=0.2):
    if X.shape[0] ==1:
        X_train = X
        X_test  = X
        y_train = y
        y_test  = y
    else:
        X_train,X_test,y_train,y_test = train_test_split(X,y,test_size = TEST_SIZE)
    return X_train,X_test,y_train,y_test

In [ ]:
def randomSplit(df_X,df_y,col_label = "lab_id",TEST_SIZE=0.2):
    ls_y_labels = list(df_y[col_label].unique())

    X_train = pd.DataFrame()
    X_test  = pd.DataFrame()
    y_train = pd.DataFrame()
    y_test  = pd.DataFrame()
    
    for label in ls_y_labels:
        y = df_y[df_y[col_label] == label]
        X = df_X.loc[y.index]
        
        Xa,Xb,ya,yb  =  quickSplit(X,y,TEST_SIZE)

        X_train = X_train.append(Xa)
        X_test  = X_test.append(Xb)
        y_train = y_train.append(ya)
        y_test  = y_test.append(yb)
    
    return X_train,X_test,y_train,y_test


In [ ]:
start = time.perf_counter()
X_train,X_validate,y_train,y_validate = randomSplit(all_features,lab_ids,TEST_SIZE=0.2)
end   = time.perf_counter()

print(end-start)

#### 351 a quick validation to check both the train and validate have covered the whole spectrum of the output lab labels.

In [ ]:
print('No of labs in training set: '+str(y_train["lab_id"].shape[0]))
print('No of labs in training set: '+str(y_validate["lab_id"].shape[0]))

print('No of labs in training set: '+str(len(list(y_train["lab_id"].unique()))))
print('No of labs in training set: '+str(len(list(y_validate["lab_id"].unique()))))

#### 352 the old way of splitting the data without validating the proportion.

X_train, X_validate, y_train, y_validate = train_test_split(all_features, lab_ids, test_size=0.30, random_state=42)

In [ ]:
# Rename our feature array
X = X_train                        # 80% of the 63,000 original Samples for 40,324

# Create our labels
y = y_train.values.ravel()

### 400 ML Model


In [ ]:
from sklearn.model_selection import GridSearchCV

#### 410 Random Forest (functional)

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_estimators =400,
                            class_weight="balanced",
                            max_depth =10,
                            bootstrap =False,
                            n_jobs = 5)
# fit our model
start = time.perf_counter()
rf.fit(X,y)
end   = time.perf_counter()

print(end-start)

#### validation evaluation

In [ ]:
start = time.perf_counter()
result= rf.score(X_validate, y_validate)
end   = time.perf_counter()

print("Single value accuracy rating:" + str(result))
print("Single value process time:"+ str(end-start))

start = time.perf_counter()
top10_result=top10_accuracy_scorer(rf, X_validate, y_validate.to_numpy())
end   = time.perf_counter()

print("Top 10 value accuracy:"+str(top10_result))
print("Top 10 value process time:"+str(end-start))

+ changing the estimator number to 400 improved the result significantly
+ increase the depth to 10 improved the result.
+ scaling the features improve the result by 4%

#### 420 Keras neuro-network (not functional yet, use with caution.)

In [ ]:
pd.DataFrame(all_features_scaled).head()

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from sklearn.model_selection import StratifiedKFold

kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=7)

mohammed = Sequential()
mohammed.add(Dense(160,activation = "relu",input_dim =159))
mohammed.add(Dense(160,activation = "relu",input_dim =159))
mohammed.add(Dense(160,activation = "relu",input_dim =159))
mohammed.add(Dense(160,activation = "relu",input_dim =159))
mohammed.add(Dense(160,activation = "relu",input_dim =159))
mohammed.add(Dense(160,activation = "relu",input_dim =159))
mohammed.add(Dense(160,activation = "relu",input_dim =159))
mohammed.add(Dense(160,activation = "relu",input_dim =159))
mohammed.compile(optimizer='rmsprop', loss='mse', metrics=['mae'])
mohammed.fit(X,y)

#### 430 X boost (functional but took a long time.)

In [ ]:
from xgboost.sklearn import XGBClassifier

xgb = XGBClassifier(n_estimators=400,learning_rate=0.05)

In [ ]:
xgb.fit(X,y)

In [ ]:
start = time.perf_counter()
result= xgb.score(X_holdout, y_holdout)
end   = time.perf_counter()

print(result)
print(end-start)

start = time.perf_counter()
top10_result=top10_accuracy_scorer(xgb, X_holdout, y_holdout.to_numpy())
end   = time.perf_counter()

print(top10_result)
print(end-start)

#### 440 MultinomialNB (functional)

In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.calibration import CalibratedClassifierCV

param_metric = {"alpha":[0.3,0.1,0.03,0.005],
               "fit_prior":[True,False]}

MNB = MultinomialNB()

GS_MNB = GridSearchCV(MNB, 
                      param_grid=param_metric,
                      scoring="accuracy",
                      n_jobs= 4,
                      verbose = 1)

In [ ]:
# cal = CalibratedClassifierCV(GS_MNB, cv=5, method="isotonic")

In [ ]:
GS_MNB.fit(X,y)

In [ ]:
start = time.perf_counter()
result= GS_MNB.score(X_validate, y_validate)
end   = time.perf_counter()

print("Single value accuracy rating:" + str(result))
print("Single value process time:"+ str(end-start))

start = time.perf_counter()
top10_result=top10_accuracy_scorer(GS_MNB, X_validate, y_validate.to_numpy())
end   = time.perf_counter()

print("Top 10 value accuracy:"+str(top10_result))
print("Top 10 value process time:"+str(end-start))

#### 450 RBF SVM (not functional yet, use with caution.)

In [ ]:
from sklearn.svm import SVC
param_metric={'kernal':('linear','rbf'),
             'C':[1,10]}

svc = SVC(C = 1,kernel = 'rbf',gamma=1)

In [ ]:
k_range = ('linear','rbf')
C_range = np.logspace(-2, 10, 13)
gamma_range = np.logspace(-9, 3, 13)
param_grid = dict(kernal = k_range,
                  gamma=gamma_range,
                  C=C_range)
param_metric = {'gamma':[0.01,0.1,1,10],
                'C':[0.01,0.1,1,10]}

In [ ]:
gsRBF = GridSearchCV(svc,
                     param_grid=param_metric,
                     scoring="accuracy",
                     n_jobs= 4,
                     verbose = 1)

In [ ]:
gsRBF.fit(X,y)

### 500 Make Predictions (untouched, only useful if we are submitting to DataDriven site.)

In [ ]:
test_values.isnull().sum()

In [ ]:
test_values_filled = test_values.fillna(0)
#test_values_filled = test_values.dropna(0)
test_values_filled.isnull().sum()

In [ ]:
#test_ngram_features = get_ngram_features(test_values, subsequences)
test_ngram_features = get_ngram_features(test_values_filled, subsequences)
all_test_features   = test_ngram_features.join(test_values_filled.drop('sequence', axis=1))

In [ ]:
###  Visualize the building blocks to "all_test_features" and "all_test_features"

#test_values.describe()

#test_ngram_features.describe()

all_test_features.describe()

In [ ]:
probas = rf.predict_proba(all_test_features)

# Examine first row
probas[0]

#### 520 Save Submission

In [ ]:
submission_format = pd.read_csv(DATA_DIR + 'submission_format_3TFRxH6.csv', index_col='sequence_id')

In [ ]:
submission_format.tail()

In [ ]:
test_values.shape

In [ ]:
probas

In [ ]:
probas.shape

In [ ]:
submission_format.shape

In [ ]:
test_values_filled.shape

In [ ]:
assert submission_format.shape == probas.shape
#assert (rf.classes_ == submission_format.columns).all()